In [149]:
#import functions and modules:
from ipywidgets import Dropdown, ColorPicker, VBox, HBox, Output, Button, Checkbox, Tab, Label
from ipywidgets import interact, interact_manual, Dropdown, SelectMultiple, HBox, VBox, Button, Output, FloatText, IntText, IntRangeSlider, RadioButtons,IntProgress, Checkbox, GridspecLayout
from IPython.display import clear_output
from IPython.core.display import display, HTML 
import datetime as dt
import pandas as pd

#for drawing the study area:
import pickle
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, GeoJSON

#map with gojson files:
  
import os
import json
import random
import requests

from ipyleaflet import Map, GeoJSON

In [175]:
def draw_studyarea(map_type='Street map'):

    
    street_map=basemap_to_tiles(basemaps.Esri.WorldStreetMap)
    
    imagery_map = basemap_to_tiles(basemaps.Esri.WorldImagery);

    
    if map_type=='Street map':

        m = Map(layers=(street_map, ), zoom=2)
    else:
        m = Map(layers=(imagery_map, ), zoom=2)
        

    draw_control = DrawControl(rectangle={'shapeOptions': {'color': '#0000FF'}})

    #add what happens when draw something
    def handle_draw(self, action, geo_json):
        
        
        list_coordinates=geo_json['geometry']['coordinates']
        
        list_coordinates=list_coordinates[0]
        
        lat=[]
        lon=[]   
        for indices in range(len(list_coordinates)):

            lon.append(list_coordinates[indices][0])
            lat.append(list_coordinates[indices][1])

        #list with five lat/lon combinations - last coordinate pair same as the first coordinate pair:
        #closes the polygon
        domain=[[min(lat), min(lon)], [max(lat), min(lon)], [max(lat), max(lon)],[min(lat), max(lon)],[min(lat), min(lon)]]


        if geo_json['geometry']['type']== 'Polygon':
            save_area = "study_area.txt"

            with open(save_area, "wb") as f:
                pickle.dump(domain, f)
            
            print('Simulation area sucessfully selected')
        else:
            print('Please draw a rectangle instead')

    draw_control.on_draw(handle_draw)
    m.add_control(draw_control)

    return m


In [185]:
def show_map_with_trajectories(domain):
  
    #this should be the name of the output geojson from running the simulation
    with open('output_trajectories_json.geojson', 'r') as f:
        data = json.load(f)
    
    lon=[]
    lat=[]
    for coordinate_pair in domain:
        lon.append(coordinate_pair[0])
        lat.append(coordinate_pair[1])
        
    center_lon=(max(lon)+min(lon))/2
    center_lat=(max(lat) + min(lat))/2
        
    m = Map(center=(center_lon, center_lat), zoom=5)

    geo_json = GeoJSON(
        data=data,
        style={
            'opacity': 1, 'Line': '9', 'fillOpacity': 0.1, 'weight': 2
        },
        hover_style={
            'color': 'yellow', 'dashArray': '0', 'fillOpacity': 0.5
        }
    )
    m.add_layer(geo_json)

    return m

In [186]:
# Create widgets for selection (ICOS data only)
def widget_ocean_modelling_option():             
        
    #what happend when change ex the start_year
    def change_yr(c):

        #selected_station = [key for (key, value) in stilt_stations.items() if value['name'] == station.value]
        

        months_int=[1,2,3,4,5,6,7,8,9,10,11,12]


        s_month.options=months_int


        #if the e_year value is smaller than the new start year value, it must be updated.
        #check if any of the options are smaller than selected start year
        updated_list=[]
        for item in s_year.options:
                                                 
            #c['new'] replaced with s_year.value 
            if item>=int(s_year.value):

                updated_list.append(item)

        #years - only after first 
        e_year.options=updated_list


    def change_mt(c):

        #the day widget populated depending on what month it is (different number of days)
        month_days_29=[2]

        month_days_30=[4,6,9,11]

        month_days_31=[1,3,5,7,8,10,12]

        if c['new'] in month_days_31:
            s_day.options=list(range(1,32))

        elif c['new'] in month_days_30:
            s_day.options=list(range(1,31))

        else:
            s_day.options=list(range(1,29))

        #when change start_month - change end month also (if same year)
        if s_year.value==e_year.value or len(e_month.options)==0:

            updated_list=[]
            for item in s_month.options:
                if item>=int(c['new']):

                    updated_list.append(item)

            e_month.options=updated_list

            
        #when change start_month - change end day also (if same year and month OR the first time)
        if s_year.value==e_year.value and s_month.value==e_month.value or len(e_day.options)==0:

            updated_list=[]
            for item in s_day.options:
                if item>=int(s_day.value):

                    updated_list.append(item)


            e_day.options=updated_list


    def change_yr_end(c):

        #changed from c['new']
        if s_year.value==e_year.value:
            
            updated_list=[]

            for item in s_month.options:

                if item>=s_month.value:
                    updated_list.append(item)

            e_month.options=updated_list

        else:

            months_int=[1,2,3,4,5,6,7,8,9,10,11]
            
   
            #get the available months for given year
            e_month.options=months_int

    def change_day(c):

        #when change the day... if the same month and year (start) - update
        if s_year.value==e_year.value and s_month.value==e_month.value:

            updated_list=[]

            for item in s_day.options:

                if item>=s_day.value:
                    updated_list.append(item)

            e_day.options=updated_list


    def change_month_end(c):

        if s_year.value==e_year.value and e_month.value==s_month.value:

            updated_list=[]

            for item in s_day.options:

                if item>=s_day.value:
                    updated_list.append(item)

            e_day.options=updated_list

        else:

            month_days_29=[2]

            month_days_30=[4,6,9,11]

            month_days_31=[1,3,5,7,8,10,12]

            if c['new'] in month_days_31:
                e_day.options=list(range(1,32))

            elif c['new'] in month_days_30:
                e_day.options=list(range(1,31))

            else:
                e_day.options=list(range(1,29))

    
                
    #make it possible to see whole name in ex the text of a radio button 
    style_bin = {'description_width': 'initial'}
    
    map_for_selection = draw_studyarea(map_type='Street map')
    
    
    list_years=list(range(1979, 2013))

    list_months=list(range(1, 12))
    #Create a Dropdown widget with year values (start year):
    s_year = Dropdown(options = list_years,
                      description = 'Start Year',
                      disabled= False,)
    
    #Create a Dropdown widget with month values (start month):
    s_month = Dropdown(options = list_months,
                       description = 'Start Month',
                       disabled= False,)
    
    #Create a Dropdown widget with year values (end year):
    e_year = Dropdown(options = list_years,
                      description = 'End Year',
                      disabled= False,)
    
    #Create a Dropdown widget with month values (end month):
    e_month = Dropdown(options = list_months,
                       description = 'End Month',
                       disabled= False,)
    
    days_january=list(range(1,32))
    s_day = Dropdown(options = days_january,
                    description = 'Start Day',
                    disabled = False,)
    
    e_day = Dropdown(options = days_january,
                description = 'End Day',
                disabled = False,)
    
    launch_interval=IntText(
                value=5,
                style=style_bin,
                description='Launch interval:',
                disabled=False)
    
    max_journey_lenght=IntText(
                value=60,
                style=style_bin,
                description='Max jounrney lengt:',
                disabled=False)
    
    vessels_tuple = [('Commercial Fishing Vessels',1),
            ('Skiffs',2),
            ('Mono-hull Sailing Vessel',3),
            ('Rustic raft with sail',4),
            ('Rustic raft without sail',5),
            ('Sea kayak', 6)]

    #Create a SelectMultiple widget (Multiselect Dropdown List):
    vessel_selection = Dropdown(options = vessels_tuple,
                              value = 5,
                              rows = 6,
                              description = 'Vessel type',
                              disabled = False)
    
    #Create a Button widget to control execution:
    update_button = Button(description='Update',
                           disabled=False,
                           button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
                           tooltip='Click me',)
    
    header_map = Output()
     
    with header_map:
        display(HTML('<p style="font-size:15px;font-weight:bold;"><br>Draw a polygon of the area where the simulaion should run: </p>'))
        

    header_date = Output()
     
    with header_date:
        display(HTML('<p style="font-size:15px;font-weight:bold;"><br>Select date range for simulations: </p>'))
        
    
    header_simulation_specifications = Output()
    
    with header_simulation_specifications:
        display(HTML('<p style="font-size:15px;font-weight:bold;"><br>Simulation specifications: </p>'))
        
    
    #NOTE vertical - start year above end year
    year_box = VBox([s_year, e_year])
    month_box = VBox([s_month, e_month])
    day_box = VBox([s_day, e_day])

    #the two vertical boxes next to each other in a horizontal box
    #Add both time-related VBoxes to a HBox:
    time_box = HBox([year_box, month_box, day_box])
    
    journey_specifications = HBox([launch_interval,max_journey_lenght])

    #Add all widgets to a VBox:
    form = VBox([header_map, map_for_selection, header_date, time_box, header_simulation_specifications,journey_specifications, vessel_selection, update_button])

    #Set font of all widgets in the form:
    time_box.layout.margin = '25px 0px 10px 0px'
    year_box.layout.margin = '0px 0px 0px 0px'
    update_button.layout.margin = '50px 0px 0px 50px' #top, right, bottom, left
    royal='#4169E1'
    update_button.style.button_color=royal

    #Initialize form output:
    form_out = Output()

    #Initialize results output widgets:
    #only use for progress bar currently
    header_output = Output()
    
    #example map output with example geojson (old trajectories):
    output_example_map = Output()

    
    #observers - what happens when change ex. change start year (s_year)
    s_year.observe(change_yr, 'value')


    s_month.observe(change_mt, 'value')


    e_year.observe(change_yr_end, 'value')


    s_day.observe(change_day, 'value')


    e_month.observe(change_month_end, 'value')
    
    
    #Define update function (happends when click the button)
    def update_func(button_c):
    
            
        start_date=dt.datetime(s_year.value,s_month.value,s_day.value,0)

        end_date=dt.datetime(e_year.value, e_month.value, e_day.value,0)
        
        date_range = pd.date_range(start_date, end_date)
        
        selected_launch_interval=launch_interval.value
        
        selected_max_journey_lenght=max_journey_lenght.value
        
        selected_vessel_type=vessel_selection.value
        
        #possibly (probably) change how the min/max lan/lons are saved (can't find in messages with Victor)
        load_area = "study_area.txt"
        
        with open(load_area, "rb") as study_area:
            domain= pickle.load(study_area)
          
        #put these into configuration file, and in functions for output:
        #print(date_range)
        #print(selected_launch_interval)
        #print(selected_max_journey_lenght)
        #print(selected_vessel_type)
        #print(domain)
    
        with header_output:
            
            clear_output()
            
            f = IntProgress(min=0, max=40, description='Loading:')

            display(f) 
            
            f.value += 1
            
        with output_example_map:
            
            clear_output()

            #takes the domain as input - to make sure the map is centered over this area
            second_map=show_map_with_trajectories(domain)

            display(second_map)
            
                
    #Call update-function when button is clicked:
    update_button.on_click(update_func)

    
    #Open form object:
    with form_out:
        
        display(form,header_output,output_example_map)
        #display(form, header_output, result_nuclear_facility_influence_by_station, result_nuclear_facility_influence_by_station_by_facility)

    #Display form:
    display(form_out)

In [188]:
widget_ocean_modelling_option()

Output()

In [42]:
start_date=dt.datetime(1979,1,1)

end_date=dt.datetime(1980,1,1)

date_range = pd.date_range(start_date, end_date, freq='5D')

print(date_range)

DatetimeIndex(['1979-01-01', '1979-01-06', '1979-01-11', '1979-01-16',
               '1979-01-21', '1979-01-26', '1979-01-31', '1979-02-05',
               '1979-02-10', '1979-02-15', '1979-02-20', '1979-02-25',
               '1979-03-02', '1979-03-07', '1979-03-12', '1979-03-17',
               '1979-03-22', '1979-03-27', '1979-04-01', '1979-04-06',
               '1979-04-11', '1979-04-16', '1979-04-21', '1979-04-26',
               '1979-05-01', '1979-05-06', '1979-05-11', '1979-05-16',
               '1979-05-21', '1979-05-26', '1979-05-31', '1979-06-05',
               '1979-06-10', '1979-06-15', '1979-06-20', '1979-06-25',
               '1979-06-30', '1979-07-05', '1979-07-10', '1979-07-15',
               '1979-07-20', '1979-07-25', '1979-07-30', '1979-08-04',
               '1979-08-09', '1979-08-14', '1979-08-19', '1979-08-24',
               '1979-08-29', '1979-09-03', '1979-09-08', '1979-09-13',
               '1979-09-18', '1979-09-23', '1979-09-28', '1979-10-03',
      